# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

City_ID           City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0        0         kodiak  57.7900 -152.4072      8.73        87         100   
1        1        bodaybo  57.8506  114.1933     24.86        60          62   
2        2  atafu village  -8.5421 -172.5159     28.66        75         100   
3        3     constantia  44.1833   28.6500     31.38        45           7   
4        4    olonkinbyen  70.9221   -8.7187      4.94        95          90   

   Wind Speed Country        Date  
0        2.06      US  1719840915  
1        1.31      RU  1719840620  
2        6.55      TK  1719840918  
3        4.52      RO  1719840709  
4        2.69      SJ  1719840920

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity", #based on humidity
    color="City" # each city has different color points for better visualization
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
null_values = city_data_df.isnull()
null_values

City_ID   City    Lat    Lng  Max Temp  Humidity  Cloudiness  Wind Speed  \
0      False  False  False  False     False     False       False       False   
1      False  False  False  False     False     False       False       False   
2      False  False  False  False     False     False       False       False   
3      False  False  False  False     False     False       False       False   
4      False  False  False  False     False     False       False       False   
..       ...    ...    ...    ...       ...       ...         ...         ...   
534    False  False  False  False     False     False       False       False   
535    False  False  False  False     False     False       False       False   
536    False  False  False  False     False     False       False       False   
537    False  False  False  False     False     False       False       False   
538    False  False  False  False     False     False       False       False   

     Country   Date  
0      False  False  
1      False  False  
2      False  False  
3      False  False  
4      False  False  
..       ...    ...  
534    False  False  
535    False  False  
536    False  False  
537    False  False  
538    False  False  

[539 rows x 10 columns]

In [6]:
#null_values.any(axis=1) # .any() is to check wheather an element along an axis is 'True'. And axis=0 means index, while axis=1 means columns.

In [16]:
# Narrow down cities that fit criteria
column_name = "Max Temp"
range_min = 22
range_max = 27
mask = (city_data_df["Max Temp"]>=range_min) & (city_data_df["Max Temp"]<=range_max)
ideal_weather = city_data_df[mask]

# Drop any rows with null values
drop_df = ideal_weather.dropna(axis=0)

# Display sample data
sample_df = drop_df.sample(n=25, replace=False).sort_index()
sample_df

City_ID              City      Lat       Lng  Max Temp  Humidity  \
16        16             wuxue  29.8506  115.5525     24.64        98   
54        54          kamakwie   9.5000  -12.2333     25.49        85   
104      104          lospalos  -8.5186  127.0025     22.53        94   
109      109      dalandzadgad  43.5708  104.4250     24.21        16   
126      126              mitu   1.1983  -70.1733     26.54        83   
133      133              nema  57.5067   50.5011     26.62        54   
146      146             kimbe  -5.5502  150.1429     24.25        94   
161      161         inhambane -23.8650   35.3833     25.99        53   
163      163           uberaba -19.7483  -47.9319     22.32        60   
166      166        fort bragg  35.1390  -79.0060     24.96        87   
190      190     narasannapeta  18.4167   84.0500     26.52        84   
208      208             eraan   9.0629  117.7016     26.29        88   
217      217  praia da vitoria  38.7333  -27.0667     23.14        73   
221      221            harper   4.3750   -7.7169     26.43        75   
241      241          tshabong -26.0500   22.4500     25.84        18   
254      254         dobryanka  58.4619   56.4165     22.15        73   
313      313            quincy  42.2529  -71.0023     22.69        68   
330      330             wewak  -3.5534  143.6268     26.36        87   
385      385         sassandra   4.9500   -6.0833     26.16        84   
396      396          sretensk  52.2453  117.7093     22.30        47   
432      432             kapit   2.0167  112.9333     23.51        99   
434      434        huntsville  34.7304  -86.5861     24.26        77   
436      436           qarauyl  48.9451   79.2550     24.69        31   
438      438        vila velha -20.3297  -40.2925     22.97        78   
527      527         manokwari  -0.8667  134.0833     26.00        78   

     Cloudiness  Wind Speed Country        Date  
16          100        2.23      CN  1719840935  
54          100        1.69      SL  1719840993  
104          98        2.83      TL  1719840916  
109           0        5.41      MN  1719841066  
126          86        0.16      CO  1719841086  
133          64        3.22      RU  1719841095  
146          98        1.82      PG  1719841113  
161          75        4.12      MZ  1719841131  
163          47        1.54      BR  1719841133  
166         100       12.35      US  1719841137  
190         100        6.39      IN  1719841170  
208         100        0.66      PH  1719841194  
217          20        3.99      PT  1719841206  
221          99        5.11      LR  1719841211  
241          35        7.18      BW  1719841238  
254          17        3.50      RU  1719840960  
313          40        6.17      US  1719841153  
330         100        3.37      PG  1719841162  
385          66        4.14      CI  1719841363  
396          90        1.00      RU  1719841449  
432         100        0.32      MY  1719841499  
434           0        4.63      US  1719841502  
436         100       10.54      KZ  1719841505  
438          75        3.09      BR  1719841507  
527          97        1.22      ID  1719841637

### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = sample_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

City Country      Lat       Lng  Humidity Hotel Name
16              wuxue      CN  29.8506  115.5525        98           
54           kamakwie      SL   9.5000  -12.2333        85           
104          lospalos      TL  -8.5186  127.0025        94           
109      dalandzadgad      MN  43.5708  104.4250        16           
126              mitu      CO   1.1983  -70.1733        83           
133              nema      RU  57.5067   50.5011        54           
146             kimbe      PG  -5.5502  150.1429        94           
161         inhambane      MZ -23.8650   35.3833        53           
163           uberaba      BR -19.7483  -47.9319        60           
166        fort bragg      US  35.1390  -79.0060        87           
190     narasannapeta      IN  18.4167   84.0500        84           
208             eraan      PH   9.0629  117.7016        88           
217  praia da vitoria      PT  38.7333  -27.0667        73           
221            harper      LR   4.3750   -7.7169        75           
241          tshabong      BW -26.0500   22.4500        18           
254         dobryanka      RU  58.4619   56.4165        73           
313            quincy      US  42.2529  -71.0023        68           
330             wewak      PG  -3.5534  143.6268        87           
385         sassandra      CI   4.9500   -6.0833        84           
396          sretensk      RU  52.2453  117.7093        47           
432             kapit      MY   2.0167  112.9333        99           
434        huntsville      US  34.7304  -86.5861        77           
436           qarauyl      KZ  48.9451   79.2550        31           
438        vila velha      BR -20.3297  -40.2925        78           
527         manokwari      ID  -0.8667  134.0833        78

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000  # Search radius in meters
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
wuxue - nearest hotel: No hotel found
kamakwie - nearest hotel: No hotel found
lospalos - nearest hotel: Hotel Roberto Carlos
dalandzadgad - nearest hotel: Хан Уул буудал
mitu - nearest hotel: Hotel Brasilia Vaupés
nema - nearest hotel: No hotel found
kimbe - nearest hotel: Kimbe Bay Hotel
inhambane - nearest hotel: Hotel de Inhambane
uberaba - nearest hotel: Hotel Metrópole
fort bragg - nearest hotel: Airborne Inn Lodging
narasannapeta - nearest hotel: No hotel found
eraan - nearest hotel: Trucut Travellers Inn
praia da vitoria - nearest hotel: Salles
harper - nearest hotel: No hotel found
tshabong - nearest hotel: No hotel found
dobryanka - nearest hotel: Уральская Венеция
quincy - nearest hotel: Howard Johnson
wewak - nearest hotel: Boutique Hotel 2
sassandra - nearest hotel: Hotel Le Pollet
sretensk - nearest hotel: No hotel found
kapit - nearest hotel: Hotel Haip Choing
huntsville - nearest hotel: 106 Jefferson Huntsville, Curio Collection by Hilton
qarauyl -

City Country      Lat       Lng  Humidity  \
16              wuxue      CN  29.8506  115.5525        98   
54           kamakwie      SL   9.5000  -12.2333        85   
104          lospalos      TL  -8.5186  127.0025        94   
109      dalandzadgad      MN  43.5708  104.4250        16   
126              mitu      CO   1.1983  -70.1733        83   
133              nema      RU  57.5067   50.5011        54   
146             kimbe      PG  -5.5502  150.1429        94   
161         inhambane      MZ -23.8650   35.3833        53   
163           uberaba      BR -19.7483  -47.9319        60   
166        fort bragg      US  35.1390  -79.0060        87   
190     narasannapeta      IN  18.4167   84.0500        84   
208             eraan      PH   9.0629  117.7016        88   
217  praia da vitoria      PT  38.7333  -27.0667        73   
221            harper      LR   4.3750   -7.7169        75   
241          tshabong      BW -26.0500   22.4500        18   
254         dobryanka      RU  58.4619   56.4165        73   
313            quincy      US  42.2529  -71.0023        68   
330             wewak      PG  -3.5534  143.6268        87   
385         sassandra      CI   4.9500   -6.0833        84   
396          sretensk      RU  52.2453  117.7093        47   
432             kapit      MY   2.0167  112.9333        99   
434        huntsville      US  34.7304  -86.5861        77   
436           qarauyl      KZ  48.9451   79.2550        31   
438        vila velha      BR -20.3297  -40.2925        78   
527         manokwari      ID  -0.8667  134.0833        78   

                                            Hotel Name  
16                                      No hotel found  
54                                      No hotel found  
104                               Hotel Roberto Carlos  
109                                     Хан Уул буудал  
126                              Hotel Brasilia Vaupés  
133                                     No hotel found  
146                                    Kimbe Bay Hotel  
161                                 Hotel de Inhambane  
163                                    Hotel Metrópole  
166                               Airborne Inn Lodging  
190                                     No hotel found  
208                              Trucut Travellers Inn  
217                                             Salles  
221                                     No hotel found  
241                                     No hotel found  
254                                  Уральская Венеция  
313                                     Howard Johnson  
330                                   Boutique Hotel 2  
385                                    Hotel Le Pollet  
396                                     No hotel found  
432                                  Hotel Haip Choing  
434  106 Jefferson Huntsville, Curio Collection by ...  
436                                     No hotel found  
438                                      Hotel Prainha  
527                                        Hotel Arfak

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="City",
    hover_cols=["City", "Country", "Hotel Name"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)